### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os, itertools, random
import wave, contextlib
import collections
from shutil import copyfile
from pydub import AudioSegment
import pandas as pd

### Paths

In [10]:
chopped = False
chopped_UBM = False
chopping_len = 1

commands_datahome = '/home/ideal_kar/Dataset/speech_commands_v0.01/'
commands_clip_datahome = '/home/ideal_kar/Dataset/speech_commands_clip_' + str(chopping_len) + 's/'
if not os.path.exists(commands_clip_datahome):
    os.makedirs(commands_clip_datahome)
    
commands_home = '/home/ideal_kar/kaldi/egs/voxceleb/s1/'
datahome = os.path.join(commands_home, 'data')

train_dir = os.path.join(datahome, 'commands_train_clip_' + str(chopping_len) + 's') if chopped_UBM else os.path.join(datahome, 'commands_train')
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    
test_dir = os.path.join(datahome, 'commands_test_clip_' + str(chopping_len) + 's') if chopped else os.path.join(datahome, 'commands_test')
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

### File Chopping

In [22]:
for path_name, dir_names, _ in os.walk(commands_datahome):
    structure = os.path.join(commands_chop_datahome, path_name[len(commands_datahome):])   
    if not os.path.isdir(structure):
        os.makedirs(structure)

for file_name in os.listdir(commands_datahome):
    full_path = os.path.join(commands_datahome, file_name)
    if (os.path.isfile(full_path)):
        copyfile(full_path, os.path.join(commands_chop_datahome, file_name))

In [23]:
for path_name, dir_names, file_names in os.walk(voxdatahome):
    structure = os.path.join(vox_chop_datahome, path_name[len(voxdatahome):])

    for dir_name in dir_names:
        for file_name in sorted(os.listdir(os.path.join(voxdatahome, dir_name))):
            try:
                sound = AudioSegment.from_file(os.path.join(path_name, dir_name, file_name))
                length = chopping_len * 1000
                chopped = sound[:length]
                chopped.export(vox_chop_datahome + dir_name + '/' + file_name, format='wav')
            except:
                sound = AudioSegment.from_file(os.path.join(path_name, dir_name, file_name))
                res = chopping_len * 1000 - len(sound)
                sound = sound + AudioSegment.silent(duration=res)
                sound.export(vox_chop_datahome + dir_name + '/' + file_name, format='wav')
                pass

### Split File Processing

In [18]:
spk2utts_train = dict()
spk2utts_test = dict()

utt2spks_dur_path_train = []
utt2spks_dur_path_test = []

targethome = commands_chop_datahome if chopped else commands_datahome

train_df = pd.read_pickle('/home/ideal_kar/PyTorch/Workspace/interspeech2018_bk/trials/commands/equal_num_102spk_iden.pkl')
test_df = pd.read_pickle('/home/ideal_kar/PyTorch/Workspace/interspeech2018_bk/trials/commands/equal_num_102spk_veri.pkl')

for index, row in train_df.iterrows():
    spk = row['spk']
    sent = row['sent']
    filename = row['file']
    
    uniqID = spk + '-' + sent + filename.split('.')[0]
    filepath = os.path.join(targethome, sent, filename)
    
    if spk not in spk2utts_train.keys():
        spk2utts_train[spk] = [uniqID]
    else:
        spk2utts_train[spk].append(uniqID)
        
    with contextlib.closing(wave.open(filepath, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        
        utt2spks_dur_path_train.append((uniqID, spk, duration, filepath))
    
for index, row in test_df.iterrows():
    spk = row['spk']
    sent = row['sent']
    filename = row['file']
    
    uniqID = spk + '-' + sent + filename.split('.')[0]
    filepath = os.path.join(targethome, sent, filename)
    
    if spk not in spk2utts_test.keys():
        spk2utts_test[spk] = [uniqID]
    else:
        spk2utts_test[spk].append(uniqID)
        
    with contextlib.closing(wave.open(filepath, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        
        utt2spks_dur_path_test.append((uniqID, spk, duration, filepath))

utt2spks_dur_path_train = sorted(utt2spks_dur_path_train, key = lambda x: x[0])
utt2spks_dur_path_test = sorted(utt2spks_dur_path_test, key = lambda x: x[0])

In [19]:
train_df.groupby(['spk']).size()

spk
00176480     4
004ae714    24
00b01445    21
00f0204f    13
012c8314    22
0135f3f2    18
014f9f65    23
01648c51    17
016e2c6d    17
019fa366    21
01b4757a    17
01bb6a2a    87
01bcfc0c    21
01d22d03    21
0227998e    22
022cd682    20
023808be     7
023a61ad    25
026290a7    19
02746d24    25
02e85b60    24
02fcd241     4
030ec18b    21
035de8fe    20
0362539c    20
037c445a    22
0397ecda    18
03c96658    19
03cf93b1    19
042186b8    21
            ..
fb24c826    16
fb2f3242    18
fb495c43     3
fb7c9b3b    21
fb7cfe0e    20
fb969eb4     1
fb9d6d23    19
fbdc07bb    20
fbe51750    17
fbf3dd31    19
fc2411fe    20
fc28c8d8    14
fc94edb0    17
fcb25a78    21
fce65496    21
fce96bac    24
fd32732a    24
fd395b74    99
fda46b78    23
fde2dee7    17
fe1916ba    18
fe291fa9    15
fe5c4a7a    18
feb1d305    23
ff21fb59    15
ff63ab0b    19
ffa76c4a    22
ffb86d3c    18
ffbb695d    21
fffcabd1    18
Length: 1766, dtype: int64

### Kaldi Stats File Writing

In [20]:
with open(train_dir + '/utt2spk', 'w') as u2s, \
open(train_dir + '/utt2dur', 'w') as u2d, \
open(train_dir + '/wav.scp', 'w') as ws:
    for uniqID, spk, dur, path in utt2spks_dur_path_train:
        u2s.write(uniqID + ' ' + spk + '\n')
        u2d.write(uniqID + ' ' + str(dur) + '\n')
        ws.write(uniqID + ' ' + path + '\n')

with open(test_dir + '/utt2spk', 'w') as u2s, \
open(test_dir + '/utt2dur', 'w') as u2d, \
open(test_dir + '/wav.scp', 'w') as ws:
    for uniqID, spk, dur, path in utt2spks_dur_path_test:
        u2s.write(uniqID + ' ' + spk + '\n')
        u2d.write(uniqID + ' ' + str(dur) + '\n')
        ws.write(uniqID + ' ' + path + '\n')

In [21]:
with open(train_dir + '/spk2utt', 'w') as s2u:
    for key in sorted(spk2utts_train.keys()):
        line = key
        for utt in sorted(spk2utts_train[key]):
            line += ' ' + utt
        s2u.write(line + '\n')
        
with open(test_dir + '/spk2utt', 'w') as s2u:
    for key in sorted(spk2utts_test.keys()):
        line = key
        for utt in sorted(spk2utts_test[key]):
            line += ' ' + utt
        s2u.write(line + '\n')